Load Julia packages (libraries)

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');
df = convert(DataFrame, d);

dcc = filter(row -> !(ismissing(row[:rgdppc_2000])), df)
dcc[:log_gdp] = log.(dcc[:rgdppc_2000])
dcc[:cont_africa] = Array{Float64}(convert(Array{Int}, dcc[:cont_africa]))
dcc[:rugged] = convert(Array{Float64}, dcc[:rugged])
first(dcc[[:rugged, :cont_africa, :log_gdp]], 5)

5×3 DataFrame
│ Row │ rugged  │ cont_africa │ log_gdp │
│     │ Float64 │ Float64     │ Float64 │
├─────┼─────────┼─────────────┼─────────┤
│ 1   │ 0.858   │ 1.0         │ 7.49261 │
│ 2   │ 3.427   │ 0.0         │ 8.21693 │
│ 3   │ 0.769   │ 0.0         │ 9.93326 │
│ 4   │ 0.775   │ 0.0         │ 9.40703 │
│ 5   │ 2.688   │ 0.0         │ 7.79234 │

Define the Stan language model

In [4]:
m_8_1 = "
data{
    int N;
    vector[N] log_gdp;
    vector[N] cont_africa;
    vector[N] rugged;
    vector[N] rugged_cont_africa;
}
parameters{
    real a;
    real bR;
    real bA;
    real bAR;
    real sigma;
}
model{
    vector[N] mu = a + bR * rugged + bA * cont_africa + bAR * rugged_cont_africa;
    sigma ~ uniform( 0 , 10 );
    bAR ~ normal( 0 , 10 );
    bA ~ normal( 0 , 10 );
    bR ~ normal( 0 , 10 );
    a ~ normal( 0 , 100 );
    log_gdp ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_8_1",
monitors = ["a", "bR", "bA", "bAR", "sigma"],
model=m_8_1, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/scripts/08/tmp/m_8_1.stan will be updated.



Input data for cmdstan

In [6]:
m_8_1_data = Dict("N" => size(dcc, 1),
"log_gdp" => dcc[:log_gdp],  "rugged" => dcc[:rugged],
"cont_africa" => dcc[:cont_africa],
"rugged_cont_africa" => dcc[:rugged] .* dcc[:cont_africa] );

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_8_1_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.22 0.14  9.00  9.46   282    1
bR    -0.21 0.08 -0.33 -0.08   275    1
bA    -1.94 0.24 -2.33 -1.59   268    1
bAR    0.40 0.14  0.18  0.62   271    1
sigma  0.96 0.05  0.87  1.04   339    1
"

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -2.20191, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/08/tmp/m_8_1.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: m_8_1_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.44, 0.44, 0.45, 0.44) seconds, 1.8 seconds total
Sampling took (0.47, 0.41, 0.44, 0.45) seconds, 1.8 seconds total

                 Mean     MCSE  StdDev     5%    50%       95%    N_Eff  N_Eff/s    R_hat
lp__              -76  4.2e-02     1.6    -79    -75  -7.4e+01  1.4e+03  8.1e+02  1.0e+00
accept_stat__    0.92  1.5e-03    0.10   0.

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.22 0.14  9.00  9.46   282    1\nbR    -0.21 0.08 -0.33 -0.08   275    1\nbA    -1.94 0.24 -2.33 -1.59   268    1\nbAR    0.40 0.14  0.18  0.62   271    1\nsigma  0.96 0.05  0.87  1.04   339    1\n"

Describe the draws

In [8]:
describe(chn)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = a, bR, bA, bAR, sigma

Empirical Posterior Estimates
─────────────────────────────────────────────
parameters
        Mean    SD   Naive SE  MCSE   ESS
    a  9.2243 0.1421   0.0022 0.0033 1000
   bA -1.9514 0.2298   0.0036 0.0044 1000
  bAR  0.3937 0.1339   0.0021 0.0026 1000
   bR -0.2027 0.0779   0.0012 0.0017 1000
sigma  0.9514 0.0506   0.0008 0.0008 1000

Quantiles
─────────────────────────────────────────────
parameters
        2.5%   25.0%   50.0%   75.0%   97.5% 
    a  8.6512  9.1266  9.2224  9.3187  9.7434
   bA -2.9812 -2.1047 -1.9550 -1.7990 -1.1157
  bAR -0.0819  0.3052  0.3955  0.4814  0.8794
   bR -0.5161 -0.2543 -0.2039 -0.1508  0.1551
sigma  0.7873  0.9160  0.9488  0.9848  1.1515



End of `m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*